In [1]:
# !pip install tensorboard

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [19]:
import pandas as pd 
import numpy as np
from torch.utils.data import dataset, DataLoader 
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import torch 

In [20]:
torch.set_default_dtype(torch.float64)

In [21]:
df = pd.read_csv("CVD_cleaned.csv")

In [22]:
features = df.loc[:, ~df.columns.isin(["Heart_Disease"])]
targets = df.loc[:,df.columns.isin(["Heart_Disease"])]

In [23]:
targets = list(targets["Heart_Disease"])

In [24]:
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, MultiLabelBinarizer
from sklearn.compose import ColumnTransformer

class MyDataset(Dataset):
    def __init__(self, features, targets, transform=None):
        self.raw_features = features 
        self.raw_targets = targets
        
        # Perform one-hot encoding on the features
        self.column_transformer = ColumnTransformer([('one_hot_encoder', OneHotEncoder(sparse=False), 
                                                        self.raw_features.select_dtypes(include=['object']).columns)],
                                                        remainder='passthrough')
        self.new_features = self.column_transformer.fit_transform(self.raw_features)
        self.label_binarizer = MultiLabelBinarizer()
        self.new_targets =  self.label_binarizer.fit_transform(self.raw_targets)
        self.transform = transform 
        
    def __len__(self):
        return len(self.raw_targets)
    
    def __getitem__(self, index):
        sample = self.new_features[index], self.new_targets[index]
        if self.transform is not None:
            sample = self.transform(sample)
        return sample
    
class to_tensor:
    def __call__(self,sample):
        features, targets= sample
        return torch.tensor(features), torch.tensor(targets)
    

In [25]:
# Example usage:
dataset = MyDataset(features, targets, transform=to_tensor())
dataloader = DataLoader(dataset, batch_size=100, shuffle=True)

In [26]:
import torch.nn as nn
# number of features (len of X cols)
input_dim = 48
# number of hidden layers
hidden_layers = 25
# number of classes (unique of y)
output_dim = 2
class Network(nn.Module):
  def __init__(self):
    super(Network, self).__init__()
    self.linear1 = nn.Linear(input_dim, hidden_layers)
    self.linear2 = nn.Linear(hidden_layers, output_dim)
  def forward(self, x):
    x = torch.sigmoid(self.linear1(x))
    x = self.linear2(x)
    return x

In [27]:
clf = Network()

In [28]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(clf.parameters(), lr=0.1)

In [29]:
epochs = 2
for epoch in range(epochs):
  running_loss = 0.0
  for i, data in enumerate(dataloader, 0):
    inputs, labels = data
    # set optimizer to zero grad to remove previous epoch gradients
    optimizer.zero_grad()
    # forward propagation
    outputs = clf(inputs)
    prediction = torch.softmax(outputs,dim=1)
    new_labels = torch.argmax(labels,dim=1)
    loss = criterion(outputs, new_labels)
    # backward propagation
    loss.backward()
    # optimize
    optimizer.step()
    running_loss += loss.item()
  # display statistics
  print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.5f}')

[1,  3089] loss: 0.43424
[2,  3089] loss: 0.43378
